# Ensemple CB recommender based on Sparse Autoencoder

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../lib')

import torch
import numpy as np
import pandas as pd

import pytorch_common.util as pu

import logging
import data as dt

import recommender as rc

<Figure size 432x288 with 0 Axes>

## Setup

In [3]:
pu.LoggerBuilder().on_console().build()

<RootLogger root (INFO)>

In [4]:
pu.set_device_name('gpu')

In [5]:
pu.get_device()

device(type='cuda', index=0)

In [6]:
torch.cuda.is_available()

True

In [7]:
torch.__version__

'1.11.0'

## Evaluación

In [8]:
def load(name): return pd.read_json(f'../datasets/movie_{name}_embedding.json')
def emb_rec(emb, name): return rc.DistanceMatrixRecommender(emb, f'{name}_embedding')

In [9]:
embeddings_by_name   = {name: load(name) for name in ['genres', 'overview', 'tags', 'title']}
recommenders_by_name = {name: emb_rec(emb, name) for name, emb in embeddings_by_name.items()}
emb_recommenders     = list(recommenders_by_name.values())

Building Distances Matrix:   0%|          | 0/18515 [00:00<?, ?it/s]

Building Distances Matrix:   0%|          | 0/18515 [00:00<?, ?it/s]

Building Distances Matrix:   0%|          | 0/18515 [00:00<?, ?it/s]

Building Distances Matrix:   0%|          | 0/18515 [00:00<?, ?it/s]

In [10]:
item_index = 10

In [11]:
recommender_group = rc.RecommenderGroup(emb_recommenders)

In [12]:
# recommender_group.recommend(item_index).show()

In [26]:
recommender = rc.EnsembleWeightedDistanceMatrixRecommender(
    emb_recommenders,
    weights = { 'tags': 0.3, 'overview': 0.1, 'genres': 0.4, 'title': 0.2}
)

In [27]:
recommender.recommend(item_index, k=50).show()


Recommender: ['genres', 'overview', 'tags', 'title']
Item


,title,image
10,Shrek 2,


Recommendations


,distance,title,image
0,0.000000,Shrek 2,
1,0.003895,Shrek,
2,0.005857,Wreck-It Ralph,
3,0.007540,Cars,
4,0.010757,Anastasia,
5,0.010906,Chitty Chitty Bang Bang,
6,0.012568,Enchanted,
7,0.012947,"Boxtrolls, The",
8,0.013048,Oblivion Island: Haruka and the Magic Mirror,
9,0.013841,"Young and Prodigious T.S. Spivet, The",
